(example-mzi)=
# Mach-Zehnder interferometer

In this tutorial, we'll define and simulate a simple circuit known as a
Mach-Zender Interferometer (MZI). This tutorial will walk through the code
found in ``examples/mzi.py`` of the Simphony repository. We expect you to have
read the previous tutorial, [intro](../intro).

In [7]:
import numpy as np
import matplotlib.pyplot as plt

## Deconstructing the MZI

In an MZI, light entering the circuit is split and travels down two paths of
differing lengths. When the light is recombined, it interferes, and the output
is frequency-dependent.

```{figure} /_static/images/mzi.png
:alt: waveguide
:align: center

A basic MZI. 
```

The MZI we'll create can be broken down into constituent parts. Simphony
includes models for these constituent parts below:

```{figure} /_static/images/mzi_outline_ports_marked.png
:alt: mzi
:align: center

Reusable components in an MZI.
```

The grating couplers are the input and output for light in the circuit. The
Y-branch can split and recombine light, and because the waveguides which carry
light across the circuit are varying length, this produces interference when
the light is recombined at the second Y-branch. We can now begin defining our
circuit in Simphony using the components we have identified.


## Defining the circuit

First we need to import the necessary Simphony modules. The ``siepic`` model
library, the ``SweepSimulator`` and the ``MonteCarloSweepSimulator`` will be
all that we need. We will also import ``matplotlib.pyplot``, from outside of
Simphony, to view the results of our simulation.

In [3]:
from simphony.libraries import siepic
from simphony.circuit import Circuit
# from simphony.simulators import MonteCarloSweepSimulator, SweepSimulator

We then create all the components identified earlier. These include the grating
couplers, the Y-branches, and the waveguides (which can be defined at any
arbitrary length, on the condition that the two lengths are different).

In [10]:
gc_input = siepic.GratingCoupler()
y_splitter = siepic.YBranch()
wg_long = siepic.Waveguide(length=150e-6)
wg_short = siepic.Waveguide(length=50e-6)
y_recombiner = siepic.YBranch()
gc_output = siepic.GratingCoupler()

Once we have our components, we can connect them. We can reference the diagram above on which connections to make.

In [11]:
# Next we connect the components to each other in a circuit
ckt = Circuit()

# You can connect two ports, models, or a model to a list of ports.
# Here we connect two ports directly:
ckt.connect(gc_input.o(1), y_splitter.o(0))

# Or, you can connect a model to a list of ports. It will iterate through
# unconnected ports of the first argument in order and pair them with the 
# ports in the list (or, if they are also models, the first unconnected port
# of each model in the list).
ckt.connect(y_splitter, [wg_short, wg_long])

# Finish making our connections
ckt.connect(gc_output.o(1), y_recombiner.o(0))
ckt.connect(y_recombiner, [wg_short, wg_long])

# Don't worry about this for now; we expose all ports to the outside world.
ckt.expose()

```{note}
There are several different ways of connecting components
in Simphony, and many are demonstrated in this example,
with comments included.

You may also wish to rename pins for better semantics when
writing your own circuit. This can be done with
``component.rename_pins('new_pin1', 'new_pin2', ...)``, on
any of the components we have created.

``rename_pins`` can be called on a model itself. This will
create default names for any new component you instantiate
of that model.
```

These are all the connections required to define our circuit. The next step will be to run the simulations.

## Sweep simulation

First, we'll do a standard sweep simulation. We define the simulator, the frequency range to run, and then connect it to
the input and output of our circuit:

In [ ]:
from simphony.simulation.classical import ClassicalSim
from simphony.simulation.simdevices import Laser, Detector

wl = np.linspace(1.5, 1.6, 1000)
laser = Laser(ckt, port=ckt.o(0), power=1)
sim = ClassicalSim(ckt=ckt, wl=wl)
result = sim.run()
wl = result.wl
out = result.output

Now we simulate. We hand the results over to [``matplotlib``](https://matplotlib.org/) for a graph visualization:

In [ ]:
f, p = simulator.simulate()
plt.plot(f, p)
plt.title("MZI")
plt.tight_layout()
plt.show()

When you run your MZI python file, it should bring up a graph showing something similar to this:

![mzi_spectrum](/_static/images/plot_mzi.png)

## Monte-Carlo simulation

Let's also run a Monte-Carlo simulation. This type of simulation estimates manufacturing variability and margin of error
found on real silicon circuits. Disconnect the previous simulator to connect the new simulator:

In [ ]:
simulator.disconnect()
simulator = MonteCarloSweepSimulator(1500e-9, 1600e-9)
simulator.multiconnect(gc_input, gc_output)

Then we run the Monte-Carlo simulation several times, plotting each curve. We will now see several, slightly different
curves on our graph due to random variation.

In [ ]:
results = simulator.simulate(runs=10)
for f, p in results:
    plt.plot(f, p)

Finally, we can plot the first simulation again with black, to make sure it displays on top. We want this because the
first simulation records results for ideal conditions.

In [ ]:
f, p = results[0]
plt.plot(f, p, "k")
plt.title("MZI Monte Carlo")
plt.tight_layout()
plt.show()

You should see something similar to this graph when you run your MZI now:

![monte carlo spectrum](/_static/images/plot_mzi_mc2.png)

We have now defined and simulated our MZI! This completes our tutorial.